In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_absolute_percentage_error
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve
from pylab import rcParams
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.impute import SimpleImputer
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import pickle
import importlib
import sys
#from visualization import plot_cv_indices
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import glob


## Adding Features

In [91]:
# Load the labeled index data from csv
labeled_index = pd.read_csv("./data/labeled_data/quarterly_labeled_index_standardized.csv")
# Concat the index data with features - Q1 1998 - Q4 2019
Marco_folder = "./data/Features_Marco"
Micro_folder = "./data/Features_Micro"
csv_file_pattern = "*.csv"
Marco_csv_files = glob.glob(f"{Marco_folder}/{csv_file_pattern}")
Micro_csv_files = glob.glob(f"{Micro_folder}/{csv_file_pattern}")


# Iterate over each CSV file in Marco_csv_files
for file in Marco_csv_files:
    data = pd.read_csv(file)
    # Get the file name from the CSV file path
    file_name = file.split('/')[-1]
    # Extract the index value from the file name
    new_name = file_name.split('.')[0]
    # Rename the 'Percentage Change' column to the index value
    data.rename(columns={'Percentage Change': new_name}, inplace=True)
    
    # Merge the data based on the specified conditions
    labeled_index = labeled_index.merge(data,
                                     how='left',
                                     left_on='Quarter',
                                     right_on='Quarter')

# Iterate over each CSV file in Micro_csv_files
for file in Micro_csv_files:
    data = pd.read_csv(file)
    # Get the file name from the CSV file path
    file_name = file.split('/')[-1]
    # Extract the index value from the file name
    new_name = file_name.split('.')[0]
    # Rename the 'Percentage Change' column to the index value
    data.rename(columns={'Percentage Change': new_name}, inplace=True)
    # Merge the data based on the specified conditions
    labeled_index = labeled_index.merge(data[['Quarter', 'index', new_name]],
                                        how='left',
                                        left_on=['Quarter', 'index'],
                                        right_on=['Quarter', 'index'])

# Save the merge_file DataFrame to a CSV file
labeled_index.to_csv("./data/merge_file.csv", index=False)
#de
print(labeled_index)

     volatility      index  crash_label  price_change  volume_change  \
0     -0.628541  000001.SS            0      0.187709      -0.127147   
1     -0.628541  000001.SS            0      0.504913      -0.127147   
2     -0.625680  000001.SS            1     -0.795446      -0.127147   
3     -0.631403  000001.SS            0     -0.843210      -0.127147   
4     -0.629515  000001.SS            0     -0.082838      -0.127147   
..          ...        ...          ...           ...            ...   
875   -0.493234      ^SSMI            0     -0.800378      -0.127147   
876   -0.529809      ^SSMI            0      0.914383      -0.127147   
877   -0.458321      ^SSMI            0      0.217264      -0.127147   
878   -0.388212      ^SSMI            0     -0.010617      -0.127147   
879   -0.351813      ^SSMI            0      0.296414      -0.127147   

           date  Quarter  Crude_Oil_Index_Excess_Return_Quarterly  \
0     31/3/1998  Q1 1998                                -0.159831 

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

## Build the model